<table align="left">

  <td>
    <a href="https://colab.research.google.com/github/leerazo/neo4j_aura_api/blob/main/02-using_aura_api/02-using_aura_api.ipynb" target="_blank">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/leerazo/neo4j_aura_api/blob/main/02-using_aura_api/02-using_aura_api.ipynb" target="_blank">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://github.com/leerazo/neo4j_aura_api/blob/main/02-using_aura_api/02-using_aura_api.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">Open in Vertex AI Workbench
    </a>
</td>
</table>

# Using the Neo4j Aura API

Let's begin by installing all the needed libraries

In [ ]:
%pip install python-dotenv

In [ ]:
from dotenv import dotenv_values
import json

## Step 2: Authenticate API and get a bearer token for access

Upload the API credential file you created earlier. If you are using Colab you can click the button below to locate and upload the file. If you are using another platform you can do this manually. 

In [ ]:
from google.colab import files
uploaded = files.upload()

Now we'll extract the credentials from the file into a dictionary for further use. 

In [ ]:
filename = list(uploaded.keys())[0]
print(filename)

aura_creds = dotenv_values(filename)
for item in aura_creds:
  print(item, '=', aura_creds[item])

Let's define a helper function which uses these credentials to generate a bearer token for access.

In [ ]:
def refresh_token(api_creds, api_base):
    api_endpoint = urljoin(api_base, '/oauth/token')

    curl_cmd = "curl --request POST '{}' --user '{}:{}' --header 'Content-Type: application/x-www-form-urlencoded' --data-urlencode 'grant_type=client_credentials'".format(api_endpoint, api_creds['CLIENT_ID'], api_creds['CLIENT_SECRET'], api_creds['CLIENT_NAME'])
    result = json.loads(subprocess.check_output(curl_cmd, shell=True))
    access_token = result['access_token']
    expires_in = result['expires_in']

    now = datetime.datetime.now()
    expiration = (now + datetime.timedelta(0, expires_in)).isoformat()

    bearer_token = {
        'access_token': access_token,
        'expiration': expiration
    }

    # Save the bearer token to a file
    token_file = 'api_bearer_token'
    with open(token_file, "w") as outfile:
        json.dump(bearer_token, outfile, indent=4)

    access_token = bearer_token['access_token']
    return access_token 

Now let's try it.

In [ ]:
api_base = 'https://api.neo4j.io/'

access_token = refresh_token(aura_creds, api_base)
print('access_token:', access_token)